In [1]:
import os
import json
import psycopg2
from tqdm import tqdm
import time
import pandas as pd
from psycopg2.extras import Json

In [ ]:
                        port=pg_credential.port,
                        user=pg_credential.username,
                        password=pg_credential.password,
                        database=pg_credential.path[1:]) # To remove slash

cursor = conn.cursor()
cursor.execute("INSERT INTO a_table (c1, c2, c3) VALUES(%s, %s, %s)", (v1, v2, v3))
cursor.close()
conn.close()

In [ ]:
cursor.close()
conn.close()

# SQL APPROACH

In [2]:
def insert_sql_func(directory, strat_index = 0):
    reconnect_count = 0 
    conn = psycopg2.connect(dbname='pgstage', user='linguist', password='eDQGK0GCStlYlHNV', host='192.168.122.183')
    cursor = conn.cursor()
    files_list = os.listdir(directory)
    for unigr_index in tqdm(range(strat_index, len(files_list))):
        unigr = files_list[unigr_index]
        if unigr.endswith("json"):
            #print(unigr)
            sim_words_list = []
            with open(os.path.join(directory,unigr)) as f:
                main_words = json.load(f)
                for main_word in main_words:
                    word = {}
                    #print(main_word['word'])
                    #main_word_id = main_word['word_id']
                    #main_ref = main_word['ref_id']
                    #main_set = main_word['setting_id']
                    #orig_word = main_word['word']
                    word['word_id'] = main_word['word_id']
                    word['ref_id'] = main_word['ref_id']
                    word['setting_id'] = main_word['setting_id']
                    word['ngramm'] = main_word['ngramm']
                    word['simlar_words'] = []
                    for similar_word in main_word['simlar_words']:
                        if similar_word['word_id'] != word['word_id'] and word['ngramm'] != similar_word['ngramm']:
                            sim_word = {}
                            sim_word['word_id'] = similar_word['word_id']
                            sim_word['ref_id'] = similar_word['ref_id']
                            sim_word['setting_id'] = similar_word['setting_id']
                            sim_word['ngramm'] = similar_word['ngramm']
                            word['simlar_words'].append(sim_word)
                        #sim_words_list.append((main_word_id, main_ref, main_set, 0, 0,0,sim_word,orig_word))
                        #print(main_word_id, main_ref, main_set, sim_word)
                        #cursor.execute("INSERT INTO linguist.mix_locales (word_id, ref_id, setting_id, mix_word_id, mix_ref_id, mix_setting_id, mix_word,orig_word) VALUES(%s, %s, %s, %s, %s, %s, %s, %s)", (main_word_id, main_ref, main_set, 0, 0,0,sim_word,orig_word))
                        #conn.commit() 
                        #break
                        
                    #print(word)
                    #print(word)
                    sim_words_list.append(word)
                    #print(len(sim_words_list))
                    #break
            #for sim_el in sim_words_list:
                #print(sim_el)
                #print()
            #break
            #print(unigr,"all words collected, going to insert")
            #cursor.executemany("INSERT INTO linguist.mix_locales (word_id, ref_id, setting_id, mix_word_id, mix_ref_id, mix_setting_id, mix_word,orig_word) VALUES(%s, %s, %s, %s, %s, %s, %s, %s)", sim_words_list)
            #conn.commit()
            #print("inseted completed")
            #print(unigr,"all words collected, going to insert") 

            try:
                cursor.execute("SELECT linguist.add_locales_ngrams( %s ::jsonb)",(Json(sim_words_list),))
                conn.commit() 
            except:
                print(unigr_index, "crashed")
                cursor.close()
                conn.close()
                reconnect_count += 1
                if reconnect_count < 30:
                    time.sleep(5)
                    conn = psycopg2.connect(dbname='pgstage', user='linguist', password='eDQGK0GCStlYlHNV', host='192.168.122.183')
                    cursor = conn.cursor()
                else:
                    print("reconnection limit")
            #print("insert completed")
        
    cursor.close()
    conn.close()
    #return sim_words_list
        
#words = insert_sql_func("./save_unigr_rus")



In [4]:
words_2 = insert_sql_func("./save_unigr_rus_2")


 22%|██▏       | 690/3187 [07:59<37:54,  1.10it/s]  

690 crashed


 52%|█████▏    | 1658/3187 [19:45<14:33,  1.75it/s] 

1658 crashed


 63%|██████▎   | 2011/3187 [24:20<13:30,  1.45it/s]  

2011 crashed


100%|██████████| 3187/3187 [38:27<00:00,  1.01it/s]  


In [10]:
insert_sql_func("./save_results_bigramm_rus")

  7%|▋         | 133/1838 [02:17<27:22,  1.04it/s]

133 crashed


 26%|██▌       | 477/1838 [08:20<21:30,  1.05it/s]  

477 crashed


 36%|███▌      | 658/1838 [12:23<23:32,  1.20s/it]  

...

 54%|█████▍    | 993/1838 [19:57<19:33,  1.39s/it]  

993 crashed


 80%|████████  | 1472/1838 [31:29<07:48,  1.28s/it]  

1472 crashed


100%|██████████| 1838/1838 [39:47<00:00,  1.02it/s]  


In [6]:
insert_sql_func("./save_results_bigramm_rus_2", strat_index = 1190)

100%|██████████| 30/30 [00:24<00:00,  1.22it/s]


In [7]:
insert_sql_func("./save_results_trigramm_rus",)

 11%|█▏        | 102/891 [01:25<10:03,  1.31it/s]

102 crashed


100%|██████████| 891/891 [12:52<00:00,  1.25it/s]  


In [4]:
insert_sql_func("./save_results_trigramm_rus_2")

 25%|██▍       | 174/704 [02:08<06:03,  1.46it/s]

174 crashed


100%|██████████| 704/704 [09:35<00:00,  1.65it/s]  


In [5]:
insert_sql_func("./save_results_qgramm_rus")

100%|██████████| 482/482 [07:21<00:00,  1.12it/s]


In [6]:
insert_sql_func("./save_results_qgramm_rus_2")

100%|██████████| 354/354 [05:01<00:00,  1.00it/s]


In [3]:
insert_sql_func("./save_unigr_tr_2")

100%|██████████| 13/13 [00:11<00:00,  1.19it/s]


In [4]:
insert_sql_func("./save_unigr_tr")

100%|██████████| 507/507 [06:00<00:00,  1.65it/s]


In [5]:
insert_sql_func("./save_bigr_tr")

100%|██████████| 187/187 [02:08<00:00,  1.08it/s]


In [6]:
insert_sql_func("./save_bigr_tr_2") 

100%|██████████| 51/51 [00:47<00:00,  1.08s/it]


In [7]:
insert_sql_func("./save_trigr_tr") 

100%|██████████| 58/58 [00:46<00:00,  1.46it/s]


In [8]:
insert_sql_func("./save_trigr_tr_2") 

100%|██████████| 15/15 [00:16<00:00,  1.30it/s]


In [9]:
insert_sql_func("./save_qgr_tr") 

100%|██████████| 33/33 [00:24<00:00,  1.54it/s]


In [ ]:
SELECT linguist.add_locales_ngrams('[
   {
       "word_id":10003175,
       "ngramm":"неопределенно угрожающий",
       "ref_id":1,
       "setting_id":1,
       "simlar_words":[
           {
               "word_id":"17720",
               "ref_id":"1",
               "setting_id":"1",
               "ngramm":"искренне неподдельно"
           },
           {
               "word_id":"21761",
               "ref_id":"1",
               "setting_id":"2",
               "ngramm":"учреждение заведение"
           }
      ]
   },
   {
       "word_id":10003491,
       "ngramm":"деревянные клинья",
       "ref_id":1,
       "setting_id":1,
       "simlar_words":[
           {
               "word_id":"24813",
               "ref_id":"1",
               "setting_id":"1",
               "ngramm":"деревянный молоток"
           },
           {
               "word_id":"53843",
               "ref_id":"1",
               "setting_id":"4",
               "ngramm":"ужасный страшный" }
        ]
}
]'::jsonb)

# CSV APPROACH

In [ ]:
def insert(directory):
    conn = psycopg2.connect(dbname='pgstage', user='linguist', password='eDQGK0GCStlYlHNV', host='192.168.122.183')
    cursor = conn.cursor()
    files_list = os.listdir(directory)
    sim_words_list = []
    for unigr_index in tqdm(range(len(files_list))):
        unigr = files_list[unigr_index]
        if unigr.endswith("json"):
            with open(os.path.join(directory,unigr)) as f:
                main_words = json.load(f)
                
                for main_word in main_words:
                    main_word_id = main_word['word_id']
                    main_ref = main_word['ref_id']
                    main_set = main_word['setting_id']
                    orig_word = main_word['word']
                    
                    for sim_word in main_word['simlar_words']:
                        sim_words_list.append((main_word_id, main_ref, main_set, 0, 0,0,sim_word,orig_word))
                        #print(main_word_id, main_ref, main_set, sim_word)
                        #cursor.execute("INSERT INTO linguist.mix_locales (word_id, ref_id, setting_id, mix_word_id, mix_ref_id, mix_setting_id, mix_word,orig_word) VALUES(%s, %s, %s, %s, %s, %s, %s, %s)", (main_word_id, main_ref, main_set, 0, 0,0,sim_word,orig_word))
                        #conn.commit() 
                        #break
            #print(unigr,"all words collected, going to insert")
            #cursor.executemany("INSERT INTO linguist.mix_locales (word_id, ref_id, setting_id, mix_word_id, mix_ref_id, mix_setting_id, mix_word,orig_word) VALUES(%s, %s, %s, %s, %s, %s, %s, %s)", sim_words_list)
            #conn.commit()
            #print("inseted completed")
                    #break
        #break
    #cursor.close()
    #conn.close()
    return sim_words_list
        
words = insert("./save_unigr_rus_2")

In [ ]:
words_first = insert("./save_unigr_rus")

In [ ]:
df_orig = pd.DataFrame(words_first, columns =['word_id', 'ref_id', 'setting_id', 
                                   'mix_word_id', 'mix_ref_id', 'mix_setting_id', 'mix_word','orig_word']) 


In [ ]:
df_orig.to_csv("unigr_2.csv", index = False)

In [ ]:
df_orig.info()

In [ ]:
for i in range(0, len(df_orig), 500000):
    print(i,i+500000)
    df = pd.DataFrame(words_first[i:i+500000], columns =['word_id', 'ref_id', 'setting_id', 
                                   'mix_word_id', 'mix_ref_id', 'mix_setting_id', 'mix_word','orig_word']) 
    df.to_csv("./unigr2(actual1)/unigr1_" + str(i) + "-" + str(i+500000) + ".csv", index = False)


In [ ]:
words[0]

In [ ]:
df_big = pd.DataFrame(words, columns =['word_id', 'ref_id', 'setting_id', 'mix_word_id', 'mix_ref_id', 'mix_setting_id', 'mix_word','orig_word']) 


In [ ]:
df_big.info()

In [ ]:
for i in range(1500000, len(df_big), 500000):
    print(i,i+500000)
    df = pd.DataFrame(words[i:i+500000], columns =['word_id', 'ref_id', 'setting_id', 
                                   'mix_word_id', 'mix_ref_id', 'mix_setting_id', 'mix_word','orig_word']) 
    df.to_csv("./unigr1(actual_2)/unigr1_" + str(i) + "-" + str(i+500000) + ".csv", index = False)


In [ ]:
df = pd.DataFrame(words[1000000:1500000], columns =['word_id', 'ref_id', 'setting_id', 
                                   'mix_word_id', 'mix_ref_id', 'mix_setting_id', 'mix_word','orig_word']) 


In [ ]:
df.to_csv("unigr1_1000-1500.csv", index = False)

In [ ]:
df.head()